In [1]:
import os
from langchain_chroma import Chroma
from langchain_community.document_loaders import WikipediaLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from model import AzureOpenAIModel

from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv(find_dotenv())

True

In [16]:
# Load wiki data
docs = WikipediaLoader(query="KP Sharma Oli", load_max_docs=2).load()

# Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=50
)

splits = text_splitter.split_documents(docs)

# # Run the hugging face embeddings on the splits
# embeddings = HuggingFaceEmbeddings(
#     model_name="all-MiniLM-L6-v2", cache_folder="./embedding_model"
# )

# OpenAI embedding
embeddings = AzureOpenAIEmbeddings(
    model=os.getenv("AZURE_EMBEDDING_DEPLOYMENT_NAME"),
    api_key=os.getenv("AZURE_EMBEDDING_KEY"),
    azure_endpoint=os.getenv("AZURE_EMBEDDING_ENDPOINT"),
)

In [25]:
# Create a vector store from the embeddings
vectorstore = Chroma.from_documents(
    documents=splits, embedding=embeddings, persist_directory="./chroma_db"
)

retriever = vectorstore.as_retriever()

### Hallucination due to outdated information

In [31]:
gpt_model = AzureOpenAIModel()

In [32]:
system_prompt = """\
You are a helpful assistant in answering about Nepal's political scene. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
"""
human_prompt = """\
Answer the following question: {question}
"""
question = "When was the last time KP Sharma Oli was elected as the Prime Minister of Nepal?"
input_message = {"question": question}

gpt_model.run_model(
    system_prompt=system_prompt,
    human_prompt=human_prompt,
    input_message=input_message,
)

'KP Sharma Oli was last elected as the Prime Minister of Nepal on February 15, 2018.'

### Grounded Generation

In [33]:
human_prompt = """\
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:
"""

question = "When was the last time KP Sharma Oli was elected as the Prime Minister of Nepal"
gpt_model.run_rag_model(
    retriever=retriever,
    question=question,
    human_prompt=human_prompt,
)

'The last time KP Sharma Oli was elected as the Prime Minister of Nepal was on 15 July 2024.'